# Pydantic AI with Ollama and MCP Server

This notebook demonstrates how to use Pydantic AI with Ollama models and MCP servers for DNA sequence analysis.

# Install dependencies
```bash
uv pip install pydantic-ai
uv pip install nest-asyncio
```

# Ollama Model is available
```bash
ollama models list
```
Please make sure qwen3:latest is available. if not, please run `ollama pull qwen3:latest`. Or choose another model and modify the model name in the code.

In [1]:
import nest_asyncio
import asyncio
import time
nest_asyncio.apply()

In [2]:
from pydantic import BaseModel
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider
from pydantic_ai.mcp import MCPServerStreamableHTTP





ollama_model = OpenAIChatModel(
    model_name='qwen3:latest',
    provider=OllamaProvider(base_url='http://localhost:11434/v1'),
)


In [3]:
# Create MCP server connection
server = MCPServerStreamableHTTP('http://localhost:8000/mcp')

In [4]:
# Create agent with MCP server tools and proper system prompt
agent_ollama = Agent(
    ollama_model, 
    toolsets=[server],
    system_prompt='''You are a DNA analysis assistant with access to specialized DNA analysis tools via MCP server.

When analyzing a DNA sequence, you should:
1. First call _list_loaded_models to see what models are available
2. Then call _dna_multi_model_predict with the DNA sequence and appropriate model names
3. Interpret and explain the results in a comprehensive way, not just only list the results

Available tools should include:
- _list_loaded_models: Lists available DNA analysis models
- _dna_multi_model_predict: Predicts DNA sequence properties using multiple models

Always use the tools to provide accurate analysis. Based on the returned results, make reasonable inferences with comprehensive biological functions of this sequence.'''
)

In [5]:
# Analyze DNA sequence using MCP server with proper async context
async def analyze_dna_sequence():
    async with agent_ollama:  # This ensures proper MCP server connection
        result = await agent_ollama.run(
            'What is the function of following DNA sequence? Please analyze it thoroughly using all available models: AGAAAAAACATGACAAGAAATCGATAATAATACAAAAGCTATGATGGTGTGCAATGTCCGTGTGCATGCGTGCACGCATTGCAACCGGCCCAAATCAAGGCCCATCGATCAGTGAATACTCATGGGCCGGCGGCCCACCACCGCTTCATCTCCTCCTCCGACGACGGGAGCACCCCCGCCGCATCGCCACCGACGAGGAGGAGGCCATTGCCGGCGGCGCCCCCGGTGAGCCGCTGCACCACGTCCCTGA'
        )
        return result

# Run the analysis
result = await analyze_dna_sequence()

time.sleep(3)

In [6]:
# Display the comprehensive analysis result
print("=== DNA Sequence Analysis Result ===")
print(result.output)
print("\n=== Usage Statistics ===")
print(result.usage())


=== DNA Sequence Analysis Result ===


The analyzed DNA sequence exhibits characteristics of a **highly conserved regulatory region** with **promoter activity** and **open chromatin structure**, suggesting it plays a critical role in gene regulation. Here's the biological interpretation:

---

### **1. Promoter Activity (Core Promoter)**
- **Model**: `promoter_model`
- **Prediction**: Labeled as **"Core promoter"** with 94% confidence.
- **Biological Significance**:
  - This sequence is likely a **transcription start site (TSS)** or a **core promoter region** critical for initiating gene transcription.
  - Core promoters typically include elements like the **TATA box**, **Nucleosome Remodeling Domain (NRD)**, or **Inr motifs**.
  - The high score suggests this region is actively involved in **RNA polymerase II recruitment** and **cis-regulatory control**.

---

### **2. Evolutionary Conservation**
- **Model**: `conservation_model`
- **Prediction**: Labeled as **"Conserved"** with 92% c

In [7]:
# Alternative approach: Test individual tool calls to debug
async def test_individual_tools():
    async with agent_ollama:
        # First, let's try to list available tools
        print("=== Testing tool availability ===")
        
        # Try to get the agent to use the list models tool
        list_result = await agent_ollama.run(
            'Please list all the available DNA analysis models using the _list_loaded_models tool.'
        )
        print("List models result:")
        print(list_result.output)
        
        return list_result

# Run individual tool test
list_result = await test_individual_tools()

=== Testing tool availability ===
List models result:


Here are the three DNA analysis models currently loaded on the server:

1. **Promoter Prediction Model**
   - **Task Type**: Binary classification
   - **Labels**: "Not promoter" vs "Core promoter"
   - **Performance**: 85% accuracy, 82% F1 score
   - **Architecture**: DNABERT (plant-specific)
   - **Memory Usage**: ~351.7MB parameters, 369.3MB total
   - **Use Case**: Identifies promoter regions in plant genomes

2. **Conservation Prediction Model**
   - **Task Type**: Binary classification
   - **Labels**: "Not conserved" vs "Conserved"
   - **Performance**: 88% accuracy, 85% F1 score
   - **Architecture**: DNABERT (plant-specific)
   - **Memory Usage**: Same as promoter model
   - **Use Case**: Detects conserved genomic regions across species

3. **Open Chromatin State Model**
   - **Task Type**: Multiclass classification
   - **Labels**: "Not open", "Full open", "Partial open"
   - **Performance**: 82% accuracy, 79% F1 score
 